In [4]:
import os
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2
import numpy as np
import sfp_nsd_utils as utils
import pandas as pd
import two_dimensional_model as model
import simulation as sim

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
params = pd.DataFrame({'sigma': [2.2], 'slope': [0.12], 'intercept': [0.35],
                       'p_1': [0.06], 'p_2': [-0.03], 'p_3': [0.07], 'p_4': [0.005],
                       'A_1': [0.04], 'A_2': [-0.01], 'A_3': [0], 'A_4': [0]})

subj_data = sim.SynthesizeRealData(sn=1, pw=True, subj_df_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/derivatives/dataframes')
subj_syn_df_2d = subj_data.synthesize_BOLD_2d(params, full_ver=True)
noisy_syn_df_2d = sim.copy_df_and_add_noise(subj_syn_df_2d, beta_col="normed_betas", noise_mean=0, noise_sd=subj_syn_df_2d['noise_SD']*1)
output_losses_history = '/Users/jh7685/Documents/Projects/test_3.csv'
output_model_history = '/Users/jh7685/Documents/Projects/test_4.csv'
output_loss_history = '/Users/jh7685/Documents/Projects/test_5.csv'



used existing dir


In [6]:
syn_dataset = model.SpatialFrequencyDataset(noisy_syn_df_2d, beta_col='normed_betas')
syn_model = model.SpatialFrequencyModel(syn_dataset.my_tensor, full_ver=True)


In [7]:
%load_ext line_profiler
%load_ext memory_profiler

In [8]:
%lprun -f  model.fit_model(syn_model, syn_dataset, learning_rate=0.01, max_epoch=1, print_every=2000, anomaly_detection=False, amsgrad=False, eps=1e-8)


**epoch no.0 loss: 79.749
**epoch no.1: Finished! final model params...
{'sigma': 0.08361691236495972, 'slope': 0.9147575497627258, 'intercept': 0.5299279093742371, 'p_1': 0.06939395517110825, 'p_2': 0.08834829926490784, 'p_3': 0.00639725849032402, 'p_4': 0.04498710483312607, 'A_1': 0.2541731297969818, 'A_2': 0.7224149107933044}
Elapsed time: 1631.65 sec


Timer unit: 1e-06 s

Total time: 1631.75 s
File: ../two_dimensional_model.py
Function: fit_model at line 363

Line #      Hits         Time  Per Hit   % Time  Line Contents
   363                                           def fit_model(model, dataset, learning_rate=1e-4, max_epoch=1000, print_every=100,
   364                                                         loss_all_voxels=True, anomaly_detection=True, amsgrad=False, eps=1e-8):
   365                                               """Fit the model. This function will allow you to run a for loop for N times set as max_epoch,
   366                                               and return the output of the training; loss history, model history."""
   367         1         14.0     14.0      0.0      torch.autograd.set_detect_anomaly(anomaly_detection)
   368                                               # [sigma, slope, intercept, p_1, p_2, p_3, p_4, A_1, A_2]
   369         1         48.0     48.0      0.0      my_parameters = [p

In [10]:
%lprun -f  model.loss_fn model.fit_model(syn_model, syn_dataset, learning_rate=0.01, max_epoch=1, print_every=2000, anomaly_detection=False, amsgrad=False, eps=1e-8)


**epoch no.0 loss: 76.459
**epoch no.1: Finished! final model params...
{'sigma': 0.10361690819263458, 'slope': 0.8947575688362122, 'intercept': 0.5099279284477234, 'p_1': 0.04939395189285278, 'p_2': 0.06834830343723297, 'p_3': -0.01360274013131857, 'p_4': 0.024987107142806053, 'A_1': 0.2741731107234955, 'A_2': 0.7024149298667908}
Elapsed time: 1751.62 sec


Timer unit: 1e-06 s

Total time: 1741.39 s
File: ../two_dimensional_model.py
Function: loss_fn at line 348

Line #      Hits         Time  Per Hit   % Time  Line Contents
   348                                           def loss_fn(voxel_info, sigma_v_info, prediction, target):
   349                                               """"""
   350         1    1374542.0 1374542.0      0.1      norm_pred = normalize(voxel_info=voxel_info, to_norm=prediction)
   351         1    1245504.0 1245504.0      0.1      norm_measured = normalize(voxel_info=voxel_info, to_norm=target)
   352         1       1577.0   1577.0      0.0      voxel_list = voxel_info.unique()
   353         1         12.0     12.0      0.0      loss_all_voxels = torch.empty(voxel_list.shape, dtype=torch.float64)
   354      3952      11023.0      2.8      0.0      for i, idx in zip(range(voxel_list.shape[0]), voxel_list):
   355      3951     408974.0    103.5      0.0          voxel_idx = voxel_info == idx
   356      3951

In [12]:
%lprun -f  model.loss_fn model.fit_model(syn_model, syn_dataset, learning_rate=0.01, max_epoch=1, print_every=2000, anomaly_detection=False, amsgrad=False, eps=1e-8)


**epoch no.0 loss: 73.833
**epoch no.1: Finished! final model params...
{'sigma': 0.12361690402030945, 'slope': 0.8747575879096985, 'intercept': 0.4899279475212097, 'p_1': 0.02939395233988762, 'p_2': 0.06834830343723297, 'p_3': -0.01360273826867342, 'p_4': 0.004987107589840889, 'A_1': 0.29417309165000916, 'A_2': 0.6824149489402771}
Elapsed time: 13.79 sec


Timer unit: 1e-06 s

Total time: 3.59898 s
File: ../two_dimensional_model.py
Function: loss_fn at line 348

Line #      Hits         Time  Per Hit   % Time  Line Contents
   348                                           def loss_fn(voxel_info, sigma_v_info, prediction, target):
   349                                               """"""
   350         1    1169434.0 1169434.0     32.5      norm_pred = normalize(voxel_info=voxel_info, to_norm=prediction)
   351         1    1100912.0 1100912.0     30.6      norm_measured = normalize(voxel_info=voxel_info, to_norm=target)
   352         1       1440.0   1440.0      0.0      voxel_list = voxel_info.unique()
   353         1         11.0     11.0      0.0      loss_all_voxels = torch.empty(voxel_list.shape, dtype=torch.float64)
   354      3952       5936.0      1.5      0.2      for i, idx in zip(range(voxel_list.shape[0]), voxel_list):
   355      3951     195383.0     49.5      5.4          voxel_idx = voxel_info == idx
   356          

In [9]:
%mprun -f model.fit_model model.fit_model(syn_model, syn_dataset, learning_rate=0.01, max_epoch=1, print_every=2000, anomaly_detection=False, amsgrad=False, eps=1e-8)


**epoch no.0 loss: 78.135
**epoch no.1: Finished! final model params...
{'sigma': 0.09361691027879715, 'slope': 0.904757559299469, 'intercept': 0.5199279189109802, 'p_1': 0.059393953531980515, 'p_2': 0.0783483013510704, 'p_3': -0.0036027412861585617, 'p_4': 0.034987106919288635, 'A_1': 0.26417312026023865, 'A_2': 0.7124149203300476}
Elapsed time: 1638.24 sec



Filename: ../two_dimensional_model.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   363   3308.8 MiB   3308.8 MiB           1   def fit_model(model, dataset, learning_rate=1e-4, max_epoch=1000, print_every=100,
   364                                                       loss_all_voxels=True, anomaly_detection=True, amsgrad=False, eps=1e-8):
   365                                             """Fit the model. This function will allow you to run a for loop for N times set as max_epoch,
   366                                             and return the output of the training; loss history, model history."""
   367   3308.8 MiB      0.0 MiB           1       torch.autograd.set_detect_anomaly(anomaly_detection)
   368                                             # [sigma, slope, intercept, p_1, p_2, p_3, p_4, A_1, A_2]
   369   3308.8 MiB      0.0 MiB          12       my_parameters = [p for p in model.parameters() if p.requires_grad]
   370                               

In [ ]:
losses_history = model.shape_losses_history(losses, syn_df)
utils.save_df_to_csv(losses_history, output_losses_history, indexing=False)
utils.save_df_to_csv(syn_model_history, output_model_history, indexing=False)
utils.save_df_to_csv(syn_loss_history, output_loss_history, indexing=False)